In [ ]:
import arcgis
from arcgis.gis import GIS
from arcgis.apps import storymap
from arcgis.mapping import WebMap
from getpass import getpass
import pandas as pd
import datetime as dt
from datetime import datetime

In [ ]:
# Create a connection to your portal for publishing (enter your ArcGIS Online 
#  password in the textbox that appears, then hit 'Enter' on your keyboard)
username = " "
gis = GIS("https://www.arcgis.com", username, getpass())

In [ ]:
find_storymaps = gis.content.search("title: Community Storymap", item_type="Web Mapping Application", max_items=500, outside_org=False)

In [ ]:
import os
today = [dt.date.today()]
log_path = r"T:\CDO_To_Esri\Storymaps\_Archive\Logfiles\Communities\UpdateCultureHistory"
log_file = os.path.join(log_path, "Update Culture History_" + str(today[0]) + str(".txt"))
out = open(log_file,'w')


for f in find_storymaps:
    print(f.title, f.id)
    try:
        if f.title != 'Alaska DCRA Community Storymaps' and not f.title.startswith('Communit') and 'Community' in str(f.title):
            csmap = storymap.JournalStoryMap(gis.content.get(str(f.id)))
            sections = csmap.properties["values"]["story"]["sections"]
            panel_ch = sections[1]
            community = f.title.replace(" Community Storymap", "").replace(" (Unincorporated Community)","")
            if community.startswith("City & Borough of "):
                community = community.replace("City & Borough of ", "")
            if community.startswith("Municipality of "):
                community = community.replace("Municipality of ","")
            query = "CommunityName = '" + str(community.replace("'","''")) + "'"
            print(query)

### CULTURE HISTORY PANEL ###
        #Access the culture and history from the culture/history feature class (IF THE PANEL ORDER CHANGES, MAKE SURE INDEX STILL APPLIES)
        ch_ft = FeatureLayer("https://maps.commerce.alaska.gov/arcgis/rest/services/Community_Related/General_CultureHistory/MapServer/0")
        culture_lookup = ch_ft.query(where=query).df
        if str(culture_lookup) != 'Empty DataFrame\nColumns: []\nIndex: []':
            culture = culture_lookup.CommunityCulture[0]
            if str(culture) == 'None':
                culture = "Information on this community's culture is unavailable."
            history = culture_lookup.CommunityHistory[0]
            if str(history) == 'None':
                history = "Information on this community's history is unavailable."
            #Populate the panel with the new culture history info
            panel_ch["media"]["webmap"]["altText"] = 'Map showing the location of ' + str(community) + ', Alaska in the context of the state and borough/REAA boundaries.'
            if str(culture) == 'None' and str(history) == 'None':
                panel_ch["content"] = '<style type="text/css">.dcra-blue {\n     display: inline-block;\n     background-color: #005e95;\n     border-color: #005e95 !important;\n     color: #fff !important;\n     padding: 0px 4px;\n     border-radius: 0px;\n}\n</style>\n<p><a class="dcra-blue" data-storymaps="MJ-ACTION-1524783785796" data-storymaps-type="navigate">Back to Table of Contents</a></p>\n\n<p>&nbsp;</p><p> No data is currently available on culture or history for this community.</p>'
            else:
                panel_ch["content"] = '<style type="text/css">.dcra-blue {\n     display: inline-block;\n     background-color: #005e95;\n     border-color: #005e95 !important;\n     color: #fff !important;\n     padding: 0px 4px;\n     border-radius: 0px;\n}\n</style>\n<p><a class="dcra-blue" data-storymaps="MJ-ACTION-1524783785796" data-storymaps-type="navigate">Back to Table of Contents</a></p>\n\n<p>&nbsp;<p>'+ history + '&nbsp;</p>\n\n<p>\n\n<p>&nbsp;<p>'+ culture + '&nbsp;</p>\n'
        else:
            panel_ch["content"] = '<style type="text/css">.dcra-blue {\n     display: inline-block;\n     background-color: #005e95;\n     border-color: #005e95 !important;\n     color: #fff !important;\n     padding: 0px 4px;\n     border-radius: 0px;\n}\n</style>\n<p><a class="dcra-blue" data-storymaps="MJ-ACTION-1524783785796" data-storymaps-type="navigate">Back to Table of Contents</a></p>\n\n<p>&nbsp;</p><p> No data is currently available on culture or history for this community.</p>'
        csmap.save()
        
        print("Culture history information updated!!")
        
    except Exception as e:
        print("----ERROR WITH " + str(community) + " STORYMAP. EXCEPTION: " + str(e) + "----")
        out.write("----ERROR WITH " + str(community) + " STORYMAP. EXCEPTION: " + str(e) + "----")
        out.write("\n\n")
        continue
        
out.close()       
print("Storymap updates complete!")